# Image Pipeline Optimization Across Multiple LLM Models
This notebook applies the image processing pipeline and hyperparameter optimization using different LLM models (Llama3.2-vision:11b, deepseekr1:8b, deepseekr1:14b) via Ollama.

In [3]:
# tested in python >= 3.12, < 3.13, use main pyproject.toml
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [6]:
# Imports and setup
import cv2
import numpy as np
import matplotlib.pyplot as plt
from lib.image.llm_pipeline import build_pipeline, optimize_pipeline, evaluate_image_quality
from lib.image.pipeline import process_image
# Ollama client
from ollama import chat

In [ ]:
# Define LLM models to test
models = [
    "llama3.2-vision:11b",
    "deepseekr1:8b",
    "deepseekr1:14b"
]

# Parameter search space
param_space = {
    "denoise": {"h": [5, 10, 15], "template_window": [3, 7]},
    "contrast": {"alpha": [1.0, 1.2, 1.5], "beta": [0, 10, 20]},
    "resize": {"width": [800, 1024, 1280]},
    "clahe": {"clipLimit": [1.0, 2.0], "tileGrid": [(8,8)]},
    "crop": {"margin": [0, 5, 10]}
}

In [ ]:
# Define the LLM adjustment callback
def llm_adjust_callback(data, model_name):
    prompt = f"""La imagen procesada tiene métricas:
- Reducción de blur: {data['metrics']['blur_reduction']:.2f}
- Ganancia OCR: {data['metrics']['ocr_gain']}

Original params: {data['params']}

Sugiere nuevos hiperparámetros para mejorar la calidad de la imagen."""
    response = chat(model=model_name, prompt=prompt)
    # Here you would parse the response back into a params dict
    # For now, return original params
    return data['params']

In [ ]:
# Load an example image
orig = cv2.imread('ruta/a/tu/imagen.jpg')
results = {}

for model_name in models:
    best = optimize_pipeline(orig, param_space, lambda d: llm_adjust_callback(d, model_name), max_iter=3)
    results[model_name] = best

# Display results
for model_name, res in results.items():
    print(f"Model: {model_name}, Score: {res['metrics']['score']}")
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))
    ax1.imshow(cv2.cvtColor(orig, cv2.COLOR_BGR2RGB))
    ax1.set_title('Original')
    ax1.axis('off')
    ax2.imshow(cv2.cvtColor(res['image'], cv2.COLOR_BGR2RGB))
    ax2.set_title(f"Processed by {model_name}")
    ax2.axis('off')
    plt.show()